In [1]:
import json
import torch
import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import DataLoader, Dataset
from transformers import GPT2Model, GPT2PreTrainedModel, GPT2Config
import pickle
import wandb
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

In [2]:
# Source: https://github.com/LCS2-IIITD/Emotion-Flip-Reasoning/blob/main/Dataloaders/nlp_utils.py
import string
import nltk
import re

numbers = {
    "0":"zero",
    "1":"one",
    "2":"two",
    "3":"three",
    "4":"four",
    "5":"five",
    "6":"six",
    "7":"seven",
    "8":"eight",
    "9":"nine"
}

def remove_puntuations(txt):
    punct = set(string.punctuation)
    txt = " ".join(txt.split("."))
    txt = " ".join(txt.split("!"))
    txt = " ".join(txt.split("?"))
    txt = " ".join(txt.split(":"))
    txt = " ".join(txt.split(";"))
    
    txt = "".join(ch for ch in txt if ch not in punct)
    return txt

def number_to_words(txt):
    for k in numbers.keys():
        txt = txt.replace(k,numbers[k]+" ")
    return txt

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'_',' ',text)
    text = number_to_words(text)
    text = remove_puntuations(text)
    text = ''.join([i if ord(i) < 128 else '' for i in text])
    text = ' '.join(text.split())
    return text

In [3]:
train_data = json.load(open('/kaggle/input/Dataset/ERC_conversational_level/train_conversation_level.json'))
val_data = json.load(open('/kaggle/input/Dataset/ERC_conversational_level/val_conversation_level.json'))

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [5]:
emotion2int = {
    'anger': 0,
    'joy': 1,
    'fear': 2,
    'disgust': 3,
    'neutral': 4,
    'surprise': 5,
    'sadness': 6
}

In [6]:
L = []
for conversation in train_data:
    for utterance in conversation['conversation']:
        L.append(emotion2int[utterance['emotion']])
class_weights=compute_class_weight(class_weight='balanced',classes=list(emotion2int.values()), y=np.array(L))

In [7]:
utterance2vec = pickle.load(open('/kaggle/input/Dataset/Embeddings/sentence_transformer_utterance2vec_768.pkl', 'rb'))

In [8]:
MAX_CONV_LEN = 35
# Defined index 7 for padding
class ERC_Dataset_Conv_Level(Dataset):
    def __init__(self, data, utterance2vec):
        self.data = data
        self.utterance2vec = utterance2vec

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        conversation = self.data[idx]['conversation']
        texts = [utterance['text'] for utterance in conversation]
        emotions = [emotion2int[utterance['emotion']] for utterance in conversation]
        text_embeddings = [torch.from_numpy(self.utterance2vec[preprocess_text(text)]) for text in texts]
                
        if(len(text_embeddings)<MAX_CONV_LEN):
            num_pads = MAX_CONV_LEN - len(text_embeddings)
            attention_mask = [1]*len(text_embeddings) + [0]*num_pads
            text_embeddings = text_embeddings + [torch.zeros(768)]*num_pads
            emotions = emotions + [7]*num_pads # 7 is the index for padding
        else:
            text_embeddings = text_embeddings[len(text_embeddings)-MAX_CONV_LEN:]
            attention_mask = [1]*MAX_CONV_LEN
            emotions = emotions[len(emotions)-MAX_CONV_LEN:]

        text_embeddings = torch.stack(text_embeddings)
        attention_mask = torch.tensor(attention_mask)
        emotions = torch.tensor(emotions)
        return {
            'text_embeddings': text_embeddings,
            'attention_mask': attention_mask,
            'emotions': emotions
        }

In [9]:
class ERC_GPT2(GPT2PreTrainedModel):
    def __init__(self, config, weights):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.GPT2 = GPT2Model(config)
        self.classifier = nn.Linear(config.n_embd, self.num_labels)
        self.weights = weights
        self.post_init()
        

    def forward(self, inputs_embeds, attention_mask, labels=None, device='cpu'):
        outputs = self.GPT2(inputs_embeds=inputs_embeds, attention_mask=attention_mask)
        outputs = outputs.last_hidden_state
        logits, label = {}, {}
        for b in range(outputs.size(0)):
            logits[b] = [self.classifier(outputs[b][i]) for i in range(outputs.size(1)) if attention_mask[b][i] == 1]
            label[b] = [labels[b][i] for i in range(outputs.size(1)) if attention_mask[b][i] == 1]
        logits = [torch.stack(logits[b]) for b in range(outputs.size(0))]
        label = [torch.stack(label[b]) for b in range(outputs.size(0))]
        loss = None
        if labels is not None:
            loss = 0
            for b in range(outputs.size(0)):
                loss += nn.CrossEntropyLoss(weight=self.weights.to(device))(logits[b].to(device), label[b].to(device))
            loss /= outputs.size(0)
        return {
            'loss': loss,
            'logits': logits,
            'labels': label   
        }

In [10]:
config = GPT2Config.from_pretrained('gpt2', num_labels=7)
model = ERC_GPT2.from_pretrained('gpt2', config=config, weights=torch.from_numpy(class_weights).float()).to(device)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of ERC_GPT2 were not initialized from the model checkpoint at gpt2 and are newly initialized: ['GPT2.h.0.attn.c_attn.bias', 'GPT2.h.0.attn.c_attn.weight', 'GPT2.h.0.attn.c_proj.bias', 'GPT2.h.0.attn.c_proj.weight', 'GPT2.h.0.ln_1.bias', 'GPT2.h.0.ln_1.weight', 'GPT2.h.0.ln_2.bias', 'GPT2.h.0.ln_2.weight', 'GPT2.h.0.mlp.c_fc.bias', 'GPT2.h.0.mlp.c_fc.weight', 'GPT2.h.0.mlp.c_proj.bias', 'GPT2.h.0.mlp.c_proj.weight', 'GPT2.h.1.attn.c_attn.bias', 'GPT2.h.1.attn.c_attn.weight', 'GPT2.h.1.attn.c_proj.bias', 'GPT2.h.1.attn.c_proj.weight', 'GPT2.h.1.ln_1.bias', 'GPT2.h.1.ln_1.weight', 'GPT2.h.1.ln_2.bias', 'GPT2.h.1.ln_2.weight', 'GPT2.h.1.mlp.c_fc.bias', 'GPT2.h.1.mlp.c_fc.weight', 'GPT2.h.1.mlp.c_proj.bias', 'GPT2.h.1.mlp.c_proj.weight', 'GPT2.h.10.attn.c_attn.bias', 'GPT2.h.10.attn.c_attn.weight', 'GPT2.h.10.attn.c_proj.bias', 'GPT2.h.10.attn.c_proj.weight', 'GPT2.h.10.ln_1.bias', 'GPT2.h.10.ln_1.weight', 'GPT2.h.10.ln_2.bias', 'GPT2.h.10.ln_2.weight', 'GPT2.h.10.mlp.c_fc.bias

In [11]:
train_dataset = ERC_Dataset_Conv_Level(train_data, utterance2vec)
val_dataset = ERC_Dataset_Conv_Level(val_data, utterance2vec)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [12]:
epochs = 100
optimizer = AdamW(model.parameters(), lr=1e-7)

In [13]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("wandb_login_key")
wandb.login(key=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [14]:
wandb.init(project='TECPEC', name='GPT2_Conv_Level', config={
    'Embedding': 'Sentence-Transformer',
    'Level': 'Conversation Level',
    'Epochs': epochs,
    'Optimizer': 'AdamW',
    'Learning Rate': 1e-7,
    'Batch Size': 16
})

wandb: Currently logged in as: shreyas21563. Use `wandb login --relogin` to force relogin


In [15]:
for epoch in range(epochs):
    model.train()
    train_pred, train_true, train_loss = [], [], 0.0
    for batch in tqdm(train_loader):
        optimizer.zero_grad()
        text_embeddings, attention_mask, emotions = batch['text_embeddings'].to(device), batch['attention_mask'].to(device), batch['emotions'].to(device)
        outputs = model(inputs_embeds=text_embeddings, attention_mask=attention_mask, labels=emotions, device=device)
        loss = outputs['loss']
        loss.backward()
        optimizer.step()
        for b in range(len(outputs['logits'])):
            train_pred.extend(torch.argmax(outputs['logits'][b], 1).tolist())
            train_true.extend(outputs['labels'][b].tolist())
        train_loss += loss.item()
    train_loss /= len(train_loader) 
    model.eval()
    val_pred, val_true, val_loss = [], [], 0.0
    with torch.no_grad():
        for batch in tqdm(val_loader):
            text_embeddings, attention_mask, emotions = batch['text_embeddings'].to(device), batch['attention_mask'].to(device), batch['emotions'].to(device)
            outputs = model(inputs_embeds=text_embeddings, attention_mask=attention_mask, labels=emotions)
            loss = outputs['loss']
            for b in range(len(outputs['logits'])):
                val_pred.extend(torch.argmax(outputs['logits'][b], 1).tolist())
                val_true.extend(outputs['labels'][b].tolist())
            val_loss += loss.item()
            
    val_loss /= len(val_loader)
    train_report = classification_report(train_true, train_pred, target_names=emotion2int.keys(), zero_division=0)
    val_report = classification_report(val_true, val_pred, target_names=emotion2int.keys(), zero_division=0)

    train_report_dict = classification_report(train_true, train_pred, target_names=emotion2int.keys(), output_dict=True, zero_division=0)
    val_report_dict = classification_report(val_true, val_pred, target_names=emotion2int.keys(), output_dict=True, zero_division=0)
    wandb.log({
        'train_loss': train_loss,
        'val_loss': val_loss,
        'train_accuracy': train_report_dict['accuracy'],
        'val_accuracy': val_report_dict['accuracy'],
        'Macro train_f1': train_report_dict['macro avg']['f1-score'],
        'Macro val_f1': val_report_dict['macro avg']['f1-score'],
        'Weighted train_f1': train_report_dict['weighted avg']['f1-score'],
        'Weighted val_f1': val_report_dict['weighted avg']['f1-score'],
    })
    print(f"Epoch: {epoch+1}, Train Loss: {train_loss}, Val Loss: {val_loss}")
    print(f"Train Report: \n{train_report}")
    print(f"Val Report: \n{val_report}")

100%|██████████| 9/9 [00:00<00:00, 12.17it/s]


Epoch: 1, Train Loss: 2.0072664526792674, Val Loss: 1.9618019527859158
Train Report: 
              precision    recall  f1-score   support

       anger       0.11      0.09      0.09      1423
         joy       0.17      0.21      0.19      2047
        fear       0.02      0.04      0.03       336
     disgust       0.02      0.02      0.02       372
     neutral       0.43      0.33      0.37      5299
    surprise       0.14      0.18      0.15      1656
     sadness       0.07      0.08      0.07      1011

    accuracy                           0.22     12144
   macro avg       0.14      0.13      0.13     12144
weighted avg       0.25      0.22      0.23     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.10      0.05      0.07       192
         joy       0.20      0.29      0.24       254
        fear       0.10      0.03      0.04        37
     disgust       0.00      0.00      0.00        42
     neutral       0.42      0.43

100%|██████████| 9/9 [00:00<00:00, 12.70it/s]


Epoch: 2, Train Loss: 1.9606205973869715, Val Loss: 1.9370386865403917
Train Report: 
              precision    recall  f1-score   support

       anger       0.11      0.08      0.09      1423
         joy       0.18      0.28      0.22      2047
        fear       0.04      0.02      0.03       336
     disgust       0.03      0.02      0.02       372
     neutral       0.44      0.39      0.41      5299
    surprise       0.15      0.18      0.16      1656
     sadness       0.09      0.08      0.09      1011

    accuracy                           0.26     12144
   macro avg       0.15      0.15      0.15     12144
weighted avg       0.27      0.26      0.26     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.14      0.09      0.11       192
         joy       0.22      0.29      0.25       254
        fear       0.20      0.03      0.05        37
     disgust       0.00      0.00      0.00        42
     neutral       0.41      0.40

100%|██████████| 9/9 [00:00<00:00, 12.59it/s]


...

Epoch: 98, Train Loss: 1.3239129812289507, Val Loss: 1.740078369776408
Train Report: 
              precision    recall  f1-score   support

       anger       0.38      0.37      0.38      1423
         joy       0.44      0.54      0.49      2047
        fear       0.20      0.47      0.28       336
     disgust       0.21      0.48      0.29       372
     neutral       0.67      0.42      0.52      5299
    surprise       0.40      0.48      0.44      1656
     sadness       0.36      0.46      0.40      1011

    accuracy                           0.45     12144
   macro avg       0.38      0.46      0.40     12144
weighted avg       0.51      0.45      0.46     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.28      0.26      0.26       192
         joy       0.40      0.57      0.47       254
        fear       0.11      0.24      0.15        37
     disgust       0.08      0.26      0.12        42
     neutral       0.57      0.30

100%|██████████| 9/9 [00:00<00:00, 12.32it/s]


Epoch: 99, Train Loss: 1.3258145130597627, Val Loss: 1.7387599415249295
Train Report: 
              precision    recall  f1-score   support

       anger       0.38      0.38      0.38      1423
         joy       0.43      0.55      0.48      2047
        fear       0.20      0.42      0.27       336
     disgust       0.19      0.46      0.27       372
     neutral       0.67      0.41      0.51      5299
    surprise       0.42      0.50      0.45      1656
     sadness       0.36      0.45      0.40      1011

    accuracy                           0.45     12144
   macro avg       0.38      0.45      0.40     12144
weighted avg       0.51      0.45      0.46     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.27      0.25      0.26       192
         joy       0.43      0.54      0.48       254
        fear       0.10      0.24      0.14        37
     disgust       0.07      0.19      0.10        42
     neutral       0.58      0.3

100%|██████████| 9/9 [00:00<00:00, 12.48it/s]

Epoch: 100, Train Loss: 1.3298427584843757, Val Loss: 1.7410512897703383
Train Report: 
              precision    recall  f1-score   support

       anger       0.36      0.36      0.36      1423
         joy       0.45      0.53      0.49      2047
        fear       0.22      0.48      0.30       336
     disgust       0.20      0.49      0.29       372
     neutral       0.66      0.41      0.50      5299
    surprise       0.42      0.50      0.46      1656
     sadness       0.33      0.46      0.39      1011

    accuracy                           0.45     12144
   macro avg       0.38      0.46      0.40     12144
weighted avg       0.50      0.45      0.46     12144

Val Report: 
              precision    recall  f1-score   support

       anger       0.27      0.26      0.26       192
         joy       0.41      0.56      0.47       254
        fear       0.11      0.24      0.15        37
     disgust       0.07      0.21      0.11        42
     neutral       0.57      0.

In [16]:
wandb.finish()

Macro train_f1,▁▁▂▂▃▃▄▄▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇████████
Macro val_f1,▁▂▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇████▇██████████████
Weighted train_f1,▁▂▃▃▄▄▅▅▅▅▆▅▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇█▇██▇███████
Weighted val_f1,▁▃▄▅▅▆▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇████▇▇██
train_accuracy,▁▂▃▃▄▄▅▅▅▅▆▅▆▆▆▆▆▇▆▆▇▇▇▇▇▇▇▇█▇██████████
train_loss,█▇▇▆▆▆▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁
val_accuracy,▁▃▃▅▅▆▅▆▆▆▆▇▇▇▇▇▆▇▇▇▇▇▇█▇▇▇▇▇▇█▇████▇▇██
val_loss,█▇▅▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▂▂▂▂▂▂▂
Macro train_f1,0.39837
Macro val_f1,0.2951
Weighted train_f1,0.45659


In [17]:
torch.save(model, "/kaggle/working/GPT2_Conv_Level.pth")